In [139]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [140]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [141]:
# Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [142]:
## Encode categorical variables (Gender)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [143]:
## OneHot Encodding na coluna Geography

onehoet_encoder_geo = OneHotEncoder()
geo_encoder = onehoet_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder, columns = onehoet_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [144]:
## Combine one hot encoder columns with the original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoder_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [145]:
## Divide the dataset into independent and dependent features
X = data.drop('EstimatedSalary', axis = 1) # Aqui foi retirado a coluna Exited
y = data['EstimatedSalary']

In [146]:
## Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [147]:
# Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [148]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehoet_encoder_geo, file)


with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Regression Problem statement

In [149]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [150]:
## Build Our ANN Model

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1], )),  ## HL1 Connected with input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1) ## Output layer
])

In [151]:
## Compile the model
# Para regressão o loss usamos mean_absolute_error diferente da classificação
# keras.io/api/losses/
model.compile(optimizer = 'adam', loss = "mean_absolute_error", metrics = ['mae'])

In [152]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 64)                832       
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [153]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [154]:
## Set up the TensorBoard
log_dir = "regression_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [155]:
## Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 20, restore_best_weights = True )

In [156]:
## Train the model
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [early_stopping_callback, tensorflow_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 100382.8672 - mae: 100382.8672 - val_loss: 98532.6172 - val_mae: 98532.6172
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99643.3984 - mae: 99643.3984 - val_loss: 97009.1797 - val_mae: 97009.1797
Epoch 3/100
250/250 [==============================] - 0s 815us/step - loss: 96959.1797 - mae: 96959.1797 - val_loss: 93059.2422 - val_mae: 93059.2422
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 91666.0469 - mae: 91666.0469 - val_loss: 86479.3594 - val_mae: 86479.3594
Epoch 5/100
250/250 [==============================] - 0s 840us/step - loss: 83936.8281 - mae: 83936.8281 - val_loss: 77896.2578 - val_mae: 77896.2578
Epoch 6/100
250/250 [==============================] - 0s 707us/step - loss: 74786.5547 - mae: 74786.5547 - val_loss: 68868.1484 - val_mae: 68868.1484
Epoch 7/100
250/250 [==============================] - 0s 799us/step - loss: 65905.6562 - mae: 659

In [157]:
## Load TensorBoard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [158]:
%tensorboard --logdir regression_logs/fit

Reusing TensorBoard on port 6006 (pid 49066), started 0:45:45 ago. (Use '!kill 49066' to kill it.)

In [159]:
# Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 470us/step - loss: 50308.3594 - mae: 50308.3594
Test MAE : 50308.359375


In [160]:
model.save('regression_model.h5')

/home/marshal/anaconda3/envs/venvANN/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
